In [ ]:
## Author- Sayan Chandra
## Roll - CS20M057
## Instructor - Mitesh M. Khapra
## Course - CS6910 (Fundamentals of Deep Learning)

In [8]:
WANDB=0

In [9]:
if WANDB:
   !pip install wandb

In [10]:
if WANDB :
  !wandb login #952756aa88ee3a472980bceb7d23632ac0a85500

In [11]:
if WANDB:
  import wandb

In [12]:
## importing all necessary modules
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

import torch
import torch.nn as cnn
import torch.optim as optimisations
from torch.nn import functional as func
from torch.utils.data import DataLoader as dataloader
import torchvision.transforms as transforms
import torchvision
from torch.autograd import Variable
import gc
from torchvision import datasets, models, transforms

In [2]:
!pip install split-folders
import splitfolders as sf

In [ ]:
!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip
!unzip nature_12K.zip

In [4]:
MAINPATH = "/content/inaturalist_12K/"
sf.fixed(MAINPATH + "train", output=MAINPATH+"train_split", seed=1337, fixed=100, oversample=False, group_prefix=None)

Copying files: 9999 files [01:18, 127.95 files/s]


In [13]:
sizew=224; sizeh=224
data_transforms = lambda inpsz : transforms.Compose([
        transforms.Resize((400, 400)),
        transforms.CenterCrop(inpsz),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [14]:
def loadTrain_Val_TestData(BATCH_SIZE, inpsz): # no arguments
  TESTPATH="/content/inaturalist_12K/val"
  train_data = torchvision.datasets.ImageFolder(root=MAINPATH+"train_split/train/", transform=data_transforms(inpsz))
  val_data = torchvision.datasets.ImageFolder(root=MAINPATH+"train_split/val/", transform=data_transforms(inpsz))
  train_data_loader = dataloader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=1)
  val_data_loader = dataloader(val_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=1)
  test_data = torchvision.datasets.ImageFolder(root=TESTPATH, transform=data_transforms(inpsz))
  test_data_loader  = dataloader(test_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=1)
  return train_data, test_data, val_data, train_data_loader, test_data_loader, val_data_loader


In [15]:
sweep_config={
    'method' : 'bayes',
    'metric' : {
        'name' : 'Val Accuracy',
        'goal' : 'maximize',
    },
    'parameters' : {
        'modelname' : {
            'values' : ['resnet50', 'alexnet', 'vgg', 'squeezenet', 'densenet', 'inceptionv3']
        },
        'epochs' : {
            'values' : [2, 4, 6],
        },
        'batch_size' : {
            'values' : [8, 16],
        },
        'learning_rate' : {
            'values' : [2.6e-3, 2e-3, 1e-4, 2e-4, 1.7e-5],
        },
        'weightdecay' : {
            'values' : [0, 0.0004, 0.00001, 0.00002]
        },
        'freeze' : {
            'values' : [0, 3, 2, 1]
        },
        'optim' : {
            'values' : ['adam', 'sgdwr']
        },
    }
}
if WANDB: sweep_id = wandb.sweep(sweep_config, entity="blackcloud", project="cs6910_dl_assignment_2") #  qv8e88tq

In [16]:
def accuracyAndLoss(data_loader, cnnModel, heyGPU, optimizer, scheduler, lossfunc) :
      predictedright=0
      totalimgs=0
      globalloss=0
      cnnModel.train(False)
      for img, y in data_loader:
          if heyGPU : img, y=Variable(img.cuda()), Variable(y.cuda())
          else : img, y=Variable(img), Variable(y)
          outp=cnnModel(img)
          curloss=lossfunc(outp, y)
          globalloss+=curloss.item()
          ignore, predicted = torch.max(outp.data, 1)
          totalimgs+=y.size(0)
          if heyGPU : predictedright+=(predicted.cpu()==y.cpu()).sum()
          else : predictedright+=(predicted==y).sum()
      curaccuracy=(predictedright/totalimgs)*100
      return curaccuracy.item(), globalloss/1000 
def trainMyModel(EPOCHS, cnnModel, train_data_loader, heyGPU, optimizer, scheduler, val_data_loader, lossfunc):
    itr=0
    for run in range(EPOCHS):
      globalloss=0
      cnnModel.train(True)
      for i, (curimg, ytrue) in enumerate(train_data_loader):
        itr+=1
        if heyGPU : curimg, ytrue=Variable(curimg.cuda()), Variable(ytrue.cuda())
        else : curimg, ytrue=Variable(curimg), Variable(ytrue)
        #print("yoo")
        optimizer.zero_grad()
        if (cnnModel.__class__.__name__=="Inception3"):
          outputt, __=cnnModel(curimg)
        else : outputt=cnnModel(curimg)
        curloss=lossfunc(outputt, ytrue)
        globalloss+=curloss.item()
        curloss.backward()
        optimizer.step()
        scheduler.step()
        #print("yooy")

      #a, b=accuracyAndLoss(train_data_loader, cnnModel, heyGPU, optimizer, scheduler, bn, lossfunc)  
      c, d=accuracyAndLoss(val_data_loader, cnnModel, heyGPU, optimizer, scheduler, lossfunc)  

      print("epochs: ",run, "Training loss: ", globalloss/8999, "val accuracy+loss ", c, d)
      if WANDB: wandb.log({"epochs":run, "Training loss":globalloss/8999, "Val Accuracy":c, "Val Loss":d})


In [17]:
def freezesome(model, _true, k):
    params=list(model.parameters())
    if _true:
        for param in params[:-k]:
            param.requires_grad = False
    return model
def initialize_model(modelname, noOfRaces=10, use_pretrained=True):
    cnnModel = None
    input_size = 224

    if modelname == "resnet50":
        cnnModel = models.resnet50(pretrained=use_pretrained)
        outputLayerClasses = cnnModel.fc.in_features
        cnnModel.fc = cnn.Linear(outputLayerClasses, noOfRaces)

    elif modelname == "alexnet":
        cnnModel = models.alexnet(pretrained=use_pretrained)
        outputLayerClasses = cnnModel.classifier[6].in_features
        cnnModel.classifier[6] = cnn.Linear(outputLayerClasses,noOfRaces)

    elif modelname == "vgg":
        cnnModel = models.vgg11_bn(pretrained=use_pretrained)
        outputLayerClasses = cnnModel.classifier[6].in_features
        cnnModel.classifier[6] = cnn.Linear(outputLayerClasses,noOfRaces)

    elif modelname == "squeezenet":
        cnnModel = models.squeezenet1_0(pretrained=use_pretrained)
        cnnModel.classifier[1] = cnn.Conv2d(512, noOfRaces, kernel_size=(1,1), stride=(1,1))
        cnnModel.num_classes = noOfRaces

    elif modelname == "densenet":
        cnnModel = models.densenet121(pretrained=use_pretrained)
        outputLayerClasses = cnnModel.classifier.in_features
        cnnModel.classifier = cnn.Linear(outputLayerClasses, noOfRaces)

    elif modelname == "inceptionv3":
        cnnModel = models.inception_v3(pretrained=use_pretrained)
        outputLayerClasses = cnnModel.AuxLogits.fc.in_features
        cnnModel.AuxLogits.fc = cnn.Linear(outputLayerClasses, noOfRaces)
        outputLayerClasses = cnnModel.fc.in_features
        cnnModel.fc = cnn.Linear(outputLayerClasses,noOfRaces)
        input_size = 299

    else:
        print("USE only THESE model NAMES : {resnet50, alexnet, vgg, squeezenet, densenet, inceptionv3}")
        exit()

    return cnnModel, input_size



In [18]:
def SweepParent():
      if WANDB:
          start=wandb.init()
          config=start.config
          cnnModel, inpsz=initialize_model(config.modelname)
          cnnModel = freezesome(cnnModel, 1, config.freeze)
          bsz=config.batch_size
          train_data, test_data, val_data, train_data_loader, test_data_loader, val_data_loader = loadTrain_Val_TestData(bsz, inpsz)
          heyGPU=torch.cuda.is_available()
          if heyGPU: cnnModel=cnnModel.cuda()
          lossfunc=cnn.CrossEntropyLoss()
          print(heyGPU)
          optim=config.optim
          if (optim=="adam") : optimizer=optimisations.Adam(cnnModel.parameters(), lr=config.learning_rate, weight_decay=config.weightdecay)
          else:  optimizer = torch.optim.SGD(cnnModel.parameters(), lr=config.learning_rate, momentum=0.92, weight_decay=config.weightdecay)
          scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=int(np.ceil(len(train_data_loader)/bsz)))
          trainMyModel(config.epochs, cnnModel, train_data_loader, heyGPU, optimizer, scheduler, val_data_loader, lossfunc)
          del cnnModel
          gc.collect()
          torch.cuda.empty_cache()

      else:
          cnnModel, inpsz=initialize_model('resnet50')
          cnnModel = freezesome(cnnModel, _true=1, k=0)
          bsz=16
          train_data, test_data, val_data, train_data_loader, test_data_loader, val_data_loader = loadTrain_Val_TestData(bsz, inpsz)
          heyGPU=torch.cuda.is_available()
          if heyGPU: cnnModel=cnnModel.cuda()
          lossfunc=cnn.CrossEntropyLoss()
          print(heyGPU)
          optimizer = torch.optim.SGD(cnnModel.parameters(), lr=0.0002, momentum=0.94, weight_decay=0.00002)
          scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=int(np.ceil(len(train_data_loader)/bsz)))
          epk=8
          trainMyModel(epk, cnnModel, train_data_loader, heyGPU, optimizer, scheduler, val_data_loader, lossfunc)
          del cnnModel
          gc.collect()
          torch.cuda.empty_cache()

In [ ]:
if WANDB: wandb.agent(sweep_id, SweepParent)
else: SweepParent()